# greenflow Hyperparameter Tuning Example

When data scientists are building machine learning models, there are a few magic numbers that are included in the model. The example numbers are depths in the tree, the learning rate, etc. The parameters that define the model architecture or training process are usually referred to as hyperparameters. They are magical because there is no good theory about what number to choose. Commonly, the hyperparameters can be searched to find a good set of them that results in the best model performance. The process of searching is referred to as hyperparameter tuning. 

There are a few popular Python-based hyperparameter tuning libraries existing: [Ray Tune](https://docs.ray.io/en/latest/tune/), [Optuna](https://optuna.org/), [HyperOpt](https://hyperopt.github.io/hyperopt/). Each library provides a set of search algorithms and schedule algorithms that is easy to use. 

Inside the greenflow, we implemented a `Context Composite Node` that can flexibly expose the hyperparameters that are interesting for tuning. The `Context Composite Node` makes hyperparameter tuning easy to do by leveraging the hyperparameter tuning libraries. Without loss of generality, we show in this tutorial an example of using `Ray Tune` to optimize the hyperparameters for an XGBoost model  

First, let's load the greenflow library

In [1]:
import sys; sys.path.insert(0, '..')
from greenflow.dataframe_flow import TaskGraph

`Ray Tune` is built on top of the library [ray](https://ray.io/), which is a distributed execution framework that makes it easy to do distributed computation. Let's setup the `ray` environment so we can utilize all the GPUs in the host node for hyperparameter search.

In [2]:
import ray
ray.init(dashboard_host='0.0.0.0')

2020-11-09 03:38:10,381	INFO resource_spec.py:231 -- Starting Ray with 53.03 GiB memory available for workers and up to 26.54 GiB for objects. You can adjust these settings with ray.init(memory=<bytes>, object_store_memory=<bytes>).
2020-11-09 03:38:11,043	INFO services.py:1193 -- View the Ray dashboard at 172.17.0.2:8265
2020-11-09 03:38:11,048	WARNING services.py:1567 -- WARNING: The object store is using /tmp instead of /dev/shm because /dev/shm has only 67100672 bytes available. This may slow down performance! You may be able to free up space by deleting files in /dev/shm or terminating any running plasma_store_server processes. If you are inside a Docker container, you may need to pass an argument with the flag '--shm-size' to 'docker run'.
2020-11-09 03:38:11,065	WARNING services.py:1567 -- WARNING: The object store is using /tmp instead of /dev/shm because /dev/shm has only 67100672 bytes available. This may slow down performance! You may be able to free up space by deleting fil

{'node_ip_address': '172.17.0.2',
 'raylet_ip_address': '172.17.0.2',
 'redis_address': '172.17.0.2:6379',
 'object_store_address': '/tmp/ray/session_2020-11-09_03-38-10_379948_6052/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2020-11-09_03-38-10_379948_6052/sockets/raylet',
 'webui_url': '172.17.0.2:8265',
 'session_dir': '/tmp/ray/session_2020-11-09_03-38-10_379948_6052'}

In the `09_greenflow_machine_leanring` notebook, we constructed a computation graph in greenflow to train a XGBoost model and run inference. We will reuse that example computation graph `xgboost_with_metrics.gq.yaml` for hyper-parameter tuning. Let's load the hyperparameter search computation graph:

In [3]:
taskGraph = TaskGraph.load_taskgraph('../taskgraphs/xgboost_example/hyper_parameters_search.gq.yaml')
taskGraph.draw()

GreenflowWidget(sub=HBox(), value=[OrderedDict([('id', 'data_gen'), ('type', 'ClassificationData'), ('conf', {'n_…

As you can see above, the `xgboost_with_metrics.gq.yaml` graph is loaded into a `Context Composite Node` with id `xgboost_model`. It exposes two input ports from the loaded graph nodes to get the training dataframe and testing dataframe. It also exposes the output ports from the loaded graph nodes to output the XGBoost training and inference results. So this composite node encapsulates the XGBoost computation graph into a single node. 

Most importantly, the context composite node can create some context parameters for the graph `xgboost_with_metrics`. The context parameter can map its value to a set of configuration parameters of the nodes inside the `xgboost_with_metrics` graph. So it is one-to-many mapping relation. When the context composite node running the computation, the values of the mapped configuration parameters will be substituted by the context parameters. Try to click on `xgboost_model` to see how to add a new context parameter and map it to a node configuration parameter.

The context composite node has a default `conf_in` port which can take a configuration object as input. Once it is connected, the context composite node will ignore its default configuration and use the fed configuration. The default `conf_out` port passes the configuration for other nodes who can take `ConfData` object. 

The `GridRandomSearchNode` in the above graph is a subclass of `Context Composite Node`, so it can take the `ConfData` from other `Context Composite Node` and expose the same input and output ports. It knows what are the context parameters defined in the `xgboost_model` node and the user can select the ones that will be used for grid and random searches. In this example, we will do a grid search for `depth` and random search for `eta`. Once the best hyperparameter is found, it will be used to do the XGBoost computation. So the outputs from `hpo` node are the XGBoost training and inference results with the best hyperparameter.

Click on the `run` button above and check the XGBoost results before and after the hyperparameter tuning. During the run the log console has informative information from the Ray Tune library. Click one the 'list' button to see the log or you can go to "View -> Show Log Console".

Since you know how hyperparameter tuning is done by greenflow, we can work on a more interesting problem. In the following graph, we will do hyperparameter tuning to find the best XGBoost model to predict the positive/negative next stock return. Note, we keep everything the same but change the dataset in the graph. 

In [4]:
taskGraph = TaskGraph.load_taskgraph('../taskgraphs/xgboost_example/xgboost_stock_hpo.gq.yaml')
taskGraph.draw()

GreenflowWidget(sub=HBox(), value=[OrderedDict([('id', ''), ('type', 'Output_Collector'), ('conf', {}), ('inputs'…

## Clean up

In [5]:
import IPython
app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}